In [10]:
# Activate multi-output in notebook
# %matplotlib inline (pour des graphes)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
from os import getcwd

DATA_DIRECTORY = getcwd() + "/../data/"

DATA_2020_PATH = DATA_DIRECTORY + "2020-communes-criteres-repartition.csv"
DATA_2021_PATH = DATA_DIRECTORY + "2021-communes-criteres-repartition.csv"

In [12]:
from scripts.load_dgcl_data import load_dgcl_file

dgcl_data_2020 = load_dgcl_file(DATA_2020_PATH)
# dgcl_data_2020.keys()

dgcl_data_2021 = load_dgcl_file(DATA_2021_PATH)
# dgcl_data_2021.keys()

Loading /Users/sch/dev/gitruc/dotations-locales/data/2020-communes-criteres-repartition.csv


/Users/sch/.local/share/virtualenvs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


Loading /Users/sch/dev/gitruc/dotations-locales/data/2021-communes-criteres-repartition.csv


/Users/sch/.local/share/virtualenvs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [13]:
filter_dgf = dgcl_data_2021["Informations générales - Population DGF Année N'"] <= 3500
filter_insee = dgcl_data_2021["Informations générales - Population INSEE Année N "] <= 3500

df_filtered = dgcl_data_2021.loc[filter_insee]

In [14]:
from pandas import merge
# dgcl_dsr_fraction_cible_2021 = dgcl_data_2021['Dotation de solidarité rurale - Cible - Montant global réparti']
# dgcl_dsu_montant_2021 = dgcl_data_2021['Dotation de solidarité urbaine - Montant total réparti']

# df_2021_2020 = dgcl_data_2021.merge(
#     dgcl_data_2020,
#    left_on="Informations générales - Nom de la commune",
#    right_on="Informations générales - Nom de la commune",
#    suffixes=['_2021','_2020'],
#    validate="many_to_many"
# ).fillna("rupture nom 2021-2020")

df_2021_2020 = merge(
    dgcl_data_2021, 
    dgcl_data_2020, 
    on=["Informations générales - Code INSEE de la commune", "Informations générales - Nom de la commune"],
    how='inner',
    suffixes=['_2021','_2020']
)

# .fillna("rupture 2021-2020")
# df_2021_2020.loc[df_2021_2020["Informations générales - Nom de la commune"] == "rupture 2021-2020"]

df_2021_2020

,Informations générales - Code INSEE de la commune,Informations générales - Nom de la commune,Informations générales - Code département de la commune_2021,Informations générales - Code SIREN de l'EPCI_2021,Informations générales - Nom de l'EPCI_2021,Informations générales - Exercice_2021,Informations générales - Régime fiscal EPCI_2021,Informations générales - Population DGF Année N'_2021,Informations générales - Population INSEE Année N _2021,Informations générales - Résidences secondaires de l'année N_2021,...,Dotation d’aménagement des communes et circonscriptions territoriales d’outre-mer - Montant total de DACOM réparti_2020,Dotation d’aménagement des communes et circonscriptions territoriales d’outre-mer - Centimes additionnels_2020,Dotation d’aménagement des communes et circonscriptions territoriales d’outre-mer - Eloignement par rapport au chef-lieu du territoire_2020,Dotation d'aménagement des communes et circonscriptions territoriales d'outre-mer - Impôts ménages_2020,Dotation d'aménagement des communes et circonscriptions territoriales d'outre-mer - Commune aurifère_2020,Dotation d'aménagement des communes et circonscriptions territoriales d'outre-mer - Indice synthétique DPOM,Dotation d'aménagement des communes et circonscriptions territoriales d'outre-mer - Coefficient de majoration de l'IS,Dotation d'aménagement des communes et circonscriptions territoriales d'outre-mer - Potentiel financier majoré de l'octroi de mer_2020,Dotation d'aménagement des communes et circonscriptions territoriales d'outre-mer - Potentiel financier majoré de l'octroi de mer par habitant_2020,Dotation d'aménagement des communes et circonscriptions territoriales d'outre-mer - Nombre de bénéficiaires du RSA
0,01001,ABERGEMENT-CLEMENCIAT,1,200069193.0,CC DE LA DOMBES,2021,FPU,805,789,16,...,0,0.0,0,0,NON,0.0,0.0,0,0.000000,0
1,01002,ABERGEMENT-DE-VAREY,1,240100883.0,CC DE LA PLAINE DE L'AIN,2021,FPU,307,254,53,...,0,0.0,0,0,NON,0.0,0.0,0,0.000000,0
2,01004,AMBERIEU-EN-BUGEY,1,240100883.0,CC DE LA PLAINE DE L'AIN,2021,FPU,14771,14586,121,...,0,0.0,0,0,NON,0.0,0.0,0,0.000000,0
3,01005,AMBERIEUX-EN-DOMBES,1,200042497.0,CC DOMBES SAONE VALLEE,2021,FPU,1757,1744,13,...,0,0.0,0,0,NON,0.0,0.0,0,0.000000,0
4,01006,AMBLEON,1,200040350.0,CC BUGEY SUD,2021,FPU,130,118,12,...,0,0.0,0,0,NON,0.0,0.0,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34868,98829,THIO,988,NaN,NaN,2021,ISO,3580,3384,196,...,421128,47290.0,17,0,NON,0.0,0.0,811803,228.548142,0
34869,98830,TOUHO,988,NaN,NaN,2021,ISO,3610,3503,107,...,519428,24734.0,46,0,NON,0.0,0.0,655394,184.566038,0
34870,98831,VOH,988,NaN,NaN,2021,ISO,3750,3610,140,...,612774,227909.0,42,0,NON,0.0,0.0,876143,231.294351,0
34871,98832,YATE,988,NaN,NaN,2021,ISO,2756,2694,62,...,475538,464729.0,12,0,NON,0.0,0.0,682938,239.207706,0


In [15]:
from pandas import DataFrame
from time import strftime

result = []
colonnes = [
    'commune',
    'code insee',
    'population dgf', 
    'population insee',
    'dsr totale 2021',
    'dsr totale 2020'
]

for v in zip(
    df_2021_2020["Informations générales - Nom de la commune"],
    df_2021_2020["Informations générales - Code INSEE de la commune"],
    df_2021_2020["Informations générales - Population DGF Année N'_2021"],
    df_2021_2020["Informations générales - Population INSEE Année N _2021"],
    (
        df_2021_2020["Dotation de solidarité rurale Bourg-centre - Montant global réparti_2021"]
        + df_2021_2020["Dotation de solidarité rurale - Péréquation - Montant global réparti (après garantie CN)_2021"]
        + df_2021_2020["Dotation de solidarité rurale - Cible - Montant global réparti_2021"]
    ),
    (
        df_2021_2020["Dotation de solidarité rurale Bourg-centre - Montant global réparti_2020"]
        + df_2021_2020["Dotation de solidarité rurale - Péréquation - Montant global réparti (après garantie CN)_2020"]
        + df_2021_2020["Dotation de solidarité rurale - Cible - Montant global réparti_2020"]
    )
):
    result.append(v)

df_result = DataFrame(result, columns=colonnes)

df_result

,commune,code insee,population dgf,population insee,dsr totale 2021,dsr totale 2020
0,ABERGEMENT-CLEMENCIAT,01001,805,789,19290,19064
1,ABERGEMENT-DE-VAREY,01002,307,254,9594,9443
2,AMBERIEU-EN-BUGEY,01004,14771,14586,0,0
3,AMBERIEUX-EN-DOMBES,01005,1757,1744,30932,30354
4,AMBLEON,01006,130,118,2879,2828
...,...,...,...,...,...,...
34868,THIO,98829,3580,3384,0,0
34869,TOUHO,98830,3610,3503,0,0
34870,VOH,98831,3750,3610,0,0
34871,YATE,98832,2756,2694,0,0


In [16]:
# est-ce une commune de moins ou pile 3500 habitants ?

df_result["<= 3_500 hab. dgf"] = df_2021_2020["Informations générales - Population DGF Année N'_2021"] <= 3_500

# est-ce une commune de moins ou pile 10000 habitants ?

df_result["<= 10_000 hab. dgf"] = df_2021_2020["Informations générales - Population DGF Année N'_2021"] <= 10_000

In [17]:
def rupture_montant_une_annee(df, nom_colonne, suffixe_an1, suffixe_an2):
    return (
        df[nom_colonne+suffixe_an2] == 0
    ) * (
        df[nom_colonne+suffixe_an1] != 0
    )
    
# pour chaque commune 2021, la DSR est-elle passée à zéro cette année-là ?

df_result["dsr bourg-centre perdue"] = rupture_montant_une_annee(
    df_2021_2020, 
    'Dotation de solidarité rurale Bourg-centre - Montant global réparti', 
    '_2020', '_2021')

df_result["dsr péréquation perdue"] = rupture_montant_une_annee(
    df_2021_2020, 
    'Dotation de solidarité rurale - Péréquation - Montant global réparti (après garantie CN)', 
    '_2020', '_2021')

df_result["dsr cible perdue"] = rupture_montant_une_annee(
    df_2021_2020, 
    'Dotation de solidarité rurale - Cible - Montant global réparti', 
    '_2020', '_2021')
# df_result.loc[df_result["dsr cible perdue"] == True]

df_result.drop_duplicates(inplace = True)
df_result

,commune,code insee,population dgf,population insee,dsr totale 2021,dsr totale 2020,<= 3_500 hab. dgf,<= 10_000 hab. dgf,dsr bourg-centre perdue,dsr péréquation perdue,dsr cible perdue
0,ABERGEMENT-CLEMENCIAT,01001,805,789,19290,19064,True,True,False,False,False
1,ABERGEMENT-DE-VAREY,01002,307,254,9594,9443,True,True,False,False,False
2,AMBERIEU-EN-BUGEY,01004,14771,14586,0,0,False,False,False,False,False
3,AMBERIEUX-EN-DOMBES,01005,1757,1744,30932,30354,True,True,False,False,False
4,AMBLEON,01006,130,118,2879,2828,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
34868,THIO,98829,3580,3384,0,0,False,True,False,False,False
34869,TOUHO,98830,3610,3503,0,0,False,True,False,False,False
34870,VOH,98831,3750,3610,0,0,False,True,False,False,False
34871,YATE,98832,2756,2694,0,0,True,True,False,False,False


In [18]:
timestamp = strftime("%Y%m%d-%H%M%S")
df_result.to_csv(
        f'dotations_communes_evol_2021-2020_{timestamp}.csv',
        index=False,
        sep=';',
        encoding='utf-8'
        )